# Домашнее задание
Использование Facial Landmarks в качестве Feature Estimator

Цель:
Создадить свой пайплайн Facial Recognition.

Описание/Пошаговая инструкция выполнения домашнего задания:
- Выберите Facial Detector по вкусу.
- Выполните Face Alignment.
- На это натравите Facial Landmarks Detector по выбору.
- На этом обучите классификатор на предпочитаемом датасете.

 # Пайплайн:
 ## __Facial Detector → Face Alignment → Facial Landmarks → Классификатор__

### 1. Выбор Facial Detector технологии
MediaPipe Face Detection - актуальный, бесплатный, функциональный, не требовательный

In [14]:
import mediapipe

### 2. Выбор Face Alignment решения
MediaPipe Face Mesh - 468 точек. Более чем достаточно для любых задач

In [15]:
face_detection = mediapipe.solutions.face_mesh # солюшн по лицу
face_mesh = face_detection.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5)

### 3. Наложение маски ~ извлечение маски


In [16]:
import cv2
import numpy as np

def get_face_landmarks(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)

    if not results.multi_face_landmarks:
        return None

    landmarks = results.multi_face_landmarks[0]
    points = np.array([[lm.x, lm.y, lm.z] for lm in landmarks.landmark])
    return points.flatten() 

In [20]:
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

X = []
y = []
label_to_name = {}

dataset_path = "Lesson 19/Homework/webcam_captures"

for idx, person_name in enumerate(os.listdir(dataset_path)):
    label_to_name[idx] = person_name
    person_dir = os.path.join(dataset_path, person_name)
    for img_file in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_file)
        img = cv2.imread(img_path)
        landmarks = get_face_landmarks(img)
        if landmarks is not None:
            X.append(landmarks)
            y.append(idx)

X = np.array(X)
y = np.array(y)

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'Lesson 19/Homework/webcam_captures'

In [10]:
# Обучение классификатора
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train)

# Сохранение модели
joblib.dump(clf, 'face_classifier.pkl')
joblib.dump(label_to_name, 'label_to_name.pkl')

# Оценка
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc * 100:.2f}%")

NameError: name 'X_train' is not defined

In [9]:
# Загрузка модели
clf = joblib.load('face_classifier.pkl')
label_to_name = joblib.load('label_to_name.pkl')

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    landmarks = get_face_landmarks(frame)
    if landmarks is not None:
        prediction = clf.predict([landmarks])
        name = label_to_name[prediction[0]]
        proba = clf.predict_proba([landmarks]).max()

        # Отображение результата
        cv2.putText(frame, f"{name} ({proba*100:.1f}%)", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'face_classifier.pkl'